In [1]:
import pandas as pd
from IPython.display import Image, HTML
from datetime import datetime

In [2]:
# convert link to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [3]:
dfs = pd.read_csv("inputs/dfs.csv")
fd = pd.read_csv("inputs/fanduel.csv")
dk = pd.read_csv("inputs/draftkings.csv")
st = pd.read_csv("inputs/statline.csv")
dfs = dfs.drop("Unnamed: 0", axis=1)
fd = fd.drop("Unnamed: 0", axis=1)
dk = dk.drop("Unnamed: 0", axis=1)
st = st.drop("Unnamed: 0", axis=1)

In [4]:
ts = datetime.now()
ts = ts.strftime("%b %d, %Y %I:%M %p")
ts = "Last Updated: " + ts

In [5]:
def highlight_dfs(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*19
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*19
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*19
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*19
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*19
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*19
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*19
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*19
    else: return ['background-color: white']*19

In [6]:
def highlight_fd(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*14
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*14
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*14
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*14
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*14
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*14
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*14
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*14
    else: return ['background-color: white']*14

In [7]:
def emojize(df):
    for i in range(len(df)):
        if df['Hot'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n🔥')
        if df['Hot'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Hot'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Cold'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n❄️')
        if df['Cold'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
        if df['Cold'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
    return df

In [8]:
def gen_page(df, out, coloring):
    games = pd.read_csv('games.csv').loc[:,['name','slug']]
    trans_games = games.set_index('name').T
    img_dict = trans_games.to_dict('list')
    images = []
    for i in df["Name"]:
        code = "https://thumbs.dreamstime.com/b/creative-illustration-default-avatar-profile-placeholder-isolated-background-art-design-grey-photo-blank-template-mockup-144849704.jpg"
        if img_dict.get(i) != None:
            code = "https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/" + img_dict.get(i)[0] + ".jpg"
        images.append(code)
    df["Picture"] = images
    
    cols = df.columns.tolist()
    cols = [cols[0]] + cols[-1:] + cols[1:-1]
    df = df[cols]
    pd.set_option('display.max_colwidth', -1)
    df.index +=1
    pre = (
        "<head>\n<script src=\"sorttable.js\"></script>\n"
        "<link rel=\"stylesheet\" href=\"df_styles.css\">\n</head>\n"
    )
    pre = pre + ts + "\n"
    pre += (
        "\n<body>\n<div id=\"filter\">\n<input type=\"text\" id=\"name\" onkeyup=\"nameSearch()\" placeholder=\"Search for name...\">\n"
        "<input type=\"text\" id=\"team\" onkeyup=\"teamSearch()\" placeholder=\"Search for team...\">\n"
        "<label for=\"pos\">Search by position:</label>\n"
        "<select id=\"pos\" onchange=\"posSearch()\" class='form-control'>\n"
        "<option></option>\n"
        "<option>PG</option>\n"
        "<option>SG</option>\n"
        "<option>SF</option>\n"
        "<option>PF</option>\n"
        "<option>C</option>\n"
        "</select>\n"
        "</div>\n\n"
    )
    
    post = (
        "\n"
        "\n<script src=\"main.js\"></script>"
        "\n</body>"
        )
    # render dataframe as html
    if coloring is not None:
        images = ['<img src="' + image + '" width="60">' for image in images]
        df["Picture"] = images
        df = emojize(df)
        body = df.style.apply(coloring, axis=1).hide_columns(['Hot', 'Cold']).render()
    else:
        html = HTML(df.to_html(escape=False ,formatters=dict(Picture=path_to_image_html)))
        body = html.data

    body = body.replace("<table id=", "<table ignore=", 1)
    body = body.replace("<table ", "<table id=\"tbl\" class=\"sortable\" ", 1)
    a = pre + body + post
    # write html to file
    with open(out, 'w', encoding="utf-8") as f:
        f.write(a)
    return df

In [9]:
gen_page(dfs, 'dfs.html', highlight_dfs)
gen_page(dfs, 'dfs.html', highlight_dfs)
gen_page(fd, 'fd.html', highlight_fd)
gen_page(dk, 'dk.html', highlight_fd)
gen_page(st, 'st.html', None)

c:\users\waldm\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.
c:\users\waldm\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\waldm\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\waldm\appdata\local\programs\python\python36-32\lib\site-pack

,Name,Picture,Position,Game,Team,Minutes,2PT FG,3PT FG,FTM,Rebounds,Assists,Blocks,Steals,Turnovers,Opponent Defensive Rank vs Position,Injury Indicator,Injury Details
1,Giannis Antetokounmpo,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/antetgi01.jpg,SF,MIL@HOU,MIL,34.63,9.95,1.61,4.72,11.73,5.61,1.24,1.25,2.26,19.0,,
2,James Harden,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/hardeja01.jpg,SG,MIL@HOU,HOU,36.40,4.65,2.69,5.18,3.78,7.31,0.06,1.67,2.81,13.0,,
3,Luka Doncic,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/doncilu01.jpg,SG,DAL@PHO,DAL,34.82,7.57,1.45,4.17,6.43,7.58,0.22,1.58,3.60,19.0,,
4,Russell Westbrook,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/westbru01.jpg,PG,MIL@HOU,HOU,34.26,8.30,0.74,3.38,5.87,4.78,0.72,1.82,2.91,18.0,,
5,Kristaps Porzingis,https://thumbs.dreamstime.com/b/creative-illustration-default-avatar-profile-placeholder-isolated-background-art-design-grey-photo-blank-template-mockup-144849704.jpg,PF,DAL@PHO,DAL,35.57,5.92,1.67,4.16,10.20,0.89,1.27,1.04,1.91,16.0,,
6,Damian Lillard,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/lillada01.jpg,PG,POR@BOS,POR,31.23,4.02,2.01,3.31,2.62,4.66,0.09,1.49,2.22,8.0,,
7,Jayson Tatum,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/tatumja01.jpg,PF,POR@BOS,BOS,26.65,3.98,1.88,3.01,6.26,2.46,0.80,1.19,1.63,29.0,,
8,Devin Booker,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/bookede01.jpg,SG,DAL@PHO,PHO,37.41,6.17,2.20,3.88,3.90,5.70,0.35,1.97,3.01,6.0,,
9,Nikola Vucevic,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/vucevni01.jpg,C,SAC@ORL,ORL,30.52,7.70,0.73,2.90,9.25,2.51,0.80,1.01,1.54,3.0,,
10,Deandre Ayton,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/aytonde01.jpg,C,DAL@PHO,PHO,34.36,7.52,0.42,2.84,10.18,1.22,1.44,1.17,2.24,22.0,,
